#Install Necessary Packages

In [ ]:
#!pip install --use-deprecated=legacy-resolver langchain openai chromadb pdfplumber beautifulsoup4 requests pyngrok langflow

# Run Langflow

In [ ]:
#!ngrok authtoken 2usKOFYirN9IGNIcDQZ9qGsvqx6_6GkRrkmoVyBjTeHBKYg4C

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""  # your actual OpenAI key


In [ ]:
from pyngrok import ngrok
import threading
import subprocess
import time

# Start Langflow as a background process
def run_langflow():
    subprocess.run(["langflow", "run"])

# Run Langflow in a separate thread
thread = threading.Thread(target=run_langflow)
thread.start()

# Wait a few seconds for Langflow to start
time.sleep(10)

# Expose Langflow using ngrok
public_url = ngrok.connect(7860).public_url
print(f"Langflow is running at: {public_url}")


#Load our Data and Analytics

In [ ]:
### Extract text from pdf

import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_text = extract_text_from_pdf("/content/group13_courseproject.pdf")


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Combine all extracted content
combined_text = pdf_text

#pdf_text + "\n\n" + csv_text + "\n\n" + article_text

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([combined_text])

# Create and store in Chroma DB
chroma_db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings()
)

chroma_db.add_documents(docs)

# Optional: persist to disk (useful across sessions)
chroma_db.persist()

print("Data stored in Chroma DB.")


#Create Chatbot Pipeline

In [ ]:
# Imports using latest LangChain (v0.1.0+) structure
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


# Load persisted Chroma DB
chroma_db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings()
)

# Create retriever
retriever = chroma_db.as_retriever()

#  Memory for multi-turn chat
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

#  Use ChatOpenAI (not OpenAI) for GPT-4, GPT-4o, GPT-3.5
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Or "gpt-3.5-turbo" if you prefer
    temperature=0
)

#  Define Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
)

# Ask a question using .invoke() instead of old call syntax
query = "What are Canada's main innovation challenges?"
response = qa_chain.invoke({"question": query})
print(response["answer"])


In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "What are the primary reasons for Canada’s decline in the Global Innovation Index over the past decade?"
response = qa_chain.invoke({"question": query})
print(response["answer"])

In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "how can Canada improve in terms of innovations?"
response = qa_chain.invoke({"question": query})
print(response["answer"])

In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "how does Canada measure among other countries in terms of innovations?"
response = qa_chain.invoke({"question": query})
print(response["answer"])

In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "How could Canada better compete with countries like China and the U.S. in AI research and development?"
response = qa_chain.invoke({"question": query})
print(response["answer"])

In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "What story does Figure 4 tell about the imbalance between R&D and marketing in Canada?"
response = qa_chain.invoke({"question": query})
print(response["answer"])

In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "How does Figure 9’s research output comparison reflect Canada’s academic strengths?"
response = qa_chain.invoke({"question": query})
print(response["answer"])

In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "What specific actions are recommended to reduce SR&ED processing times"
response = qa_chain.invoke({"question": query})
print(response["answer"])

In [ ]:
# Ask a question using .invoke() instead of old call syntax
query = "Why does the report suggest placing SR&ED consultants directly in innovation hubs?"
response = qa_chain.invoke({"question": query})
print(response["answer"])